In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import latentscope as ls

In [2]:
dataset = load_dataset("aintech/vdf_20240125_130746_ac5a6_medium_articles")

In [3]:
df = dataset["train"].to_pandas()

In [4]:
df.columns

Index(['id', 'title', 'title_vector', 'link', 'reading_time', 'publication',
       'claps', 'responses', 'vector_text-embedding-3-small'],
      dtype='object')

In [5]:
df.head()


,id,title,title_vector,link,reading_time,publication,claps,responses,vector_text-embedding-3-small
0,0,The Reported Mortality Rate of Coronavirus Is ...,"[0.041732933, 0.013779674, -0.027564144, -0.01...",https://medium.com/swlh/the-reported-mortality...,13,The Startup,1100,18,"[0.01815902255475521, -0.000556989514734596, 0..."
1,1,Dashboards in Python: 3 Advanced Examples for ...,"[0.0039737443, 0.003020432, -0.0006188639, 0.0...",https://medium.com/swlh/dashboards-in-python-3...,14,The Startup,726,3,"[-0.017228988930583, -0.03712876886129379, 0.0..."
2,2,How Can We Best Switch in Python?,"[0.031961977, 0.00047043373, -0.018263113, 0.0...",https://medium.com/swlh/how-can-we-best-switch...,6,The Startup,500,7,"[-0.0027948690112680197, -0.006100953556597233..."
3,3,Maternity leave shouldn’t set women back,"[0.032572296, -0.011148319, -0.01688577, -0.00...",https://medium.com/swlh/maternity-leave-should...,9,The Startup,460,1,"[0.04098882898688316, 0.054017066955566406, 0...."
4,4,Python NLP Tutorial: Information Extraction an...,"[-0.011735886, -0.016938083, -0.027233299, 0.0...",https://medium.com/swlh/python-nlp-tutorial-in...,7,The Startup,163,0,"[-0.0338607020676136, 0.005243391264230013, -0..."


In [6]:
# embeddings = df["title_vector"].to_numpy()
embeddings = df["vector_text-embedding-3-small"].to_numpy()

In [7]:
embeddings.shape

(5979,)

In [8]:
# Convert the numpy array of lists into a numpy array of numpy arrays
embeddings = np.array([np.array(embedding) for embedding in embeddings])


In [9]:
embeddings.shape

(5979, 1536)

In [10]:
# Copy the dataframe excluding the vector columns
df2 = df.drop(columns=["title_vector", "vector_text-embedding-3-small"])

In [11]:
ls.ingest("medium_articles", df2, "title")

Loading environment variables from: /Users/dhruvanand/Code/vector-io/.env
DATA DIR ./
DIRECTORY ./medium_articles
   id                                              title  \
0   0  The Reported Mortality Rate of Coronavirus Is ...   
1   1  Dashboards in Python: 3 Advanced Examples for ...   
2   2                  How Can We Best Switch in Python?   
3   3           Maternity leave shouldn’t set women back   
4   4  Python NLP Tutorial: Information Extraction an...   

                                                link  reading_time  \
0  https://medium.com/swlh/the-reported-mortality...            13   
1  https://medium.com/swlh/dashboards-in-python-3...            14   
2  https://medium.com/swlh/how-can-we-best-switch...             6   
3  https://medium.com/swlh/maternity-leave-should...             9   
4  https://medium.com/swlh/python-nlp-tutorial-in...             7   

   publication  claps  responses  
0  The Startup   1100         18  
1  The Startup    726          3  

In [12]:
# %pip install latentscope -U

In [14]:
%pip show latentscope

/Users/dhruvanand/miniforge3/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Name: latentscope
Version: 0.1.2
Summary: Quickly embed, project, cluster and explore a dataset.
Home-page: https://github.com/enjalot/latent-scope
Author: 
Author-email: 
License: 
Location: /Users/dhruvanand/miniforge3/lib/python3.10/site-packages
Requires: accelerate, aiohttp, aiolimiter, aiosignal, annotated-types, anyio, appnope, argon2-cffi, argon2-cffi-bindings, arrow, asttokens, async-lru, attrs, Babel, backoff, beautifulsoup4, bleach, blinker, certifi, cffi, charset-normalizer, click, cohere, comm, contourpy, cycler, Cython, debugpy, decorator, defusedxml, distro, einops, executing, fastavro, fastjsonschema, filelock, Flask, Flask-Cors, fonttools, fqdn, frozenlist, fsspec, h11, h5py, hdbscan, httpcore, httpx, huggingface-hub, idna, importlib-metadata, ipykernel, ipython, ipywidgets, isoduration, itsdangerous, jedi, Jinja2, joblib, json5, jsonpointer, jsonschema, jsonschema-specifications, jupyter, jupyter-client, jupyter-console, jupyter-core, jupyter-events, jupyter-lsp, jupy

In [15]:

ls.import_embeddings("medium_articles", embeddings, text_column="title", model_id="openai-text-embedding-3-small")

AttributeError: module 'latentscope' has no attribute 'import_embeddings'

In [ ]:

embeddings2 = df["title_vector"].to_numpy()
embeddings2 = np.array([np.array(embedding) for embedding in embeddings2])

In [ ]:
# the model (facebook/dpr-ctx_encoder-single-nq-base) isn't in our supported list yet. 
# we can still import the embeddings, but we won't be able to use the model for similarity search
ls.import_embeddings("medium_articles", embeddings2, text_column="title", model_id="") 